# Results Analysis for Hindi BabyLM

This notebook analyzes experimental results:
- Training curves comparison
- Evaluation metrics visualization  
- Statistical significance testing
- Model comparison
- Thesis figure generation

In [ ]:
import sys
import os
from pathlib import Path

# Add project root to Python path - multiple fallback strategies
notebook_dir = Path(__file__).parent if '__file__' in globals() else Path.cwd()

# Try to find project root by looking for key directories
current = notebook_dir
while current != current.parent:
    if (current / 'src').exists() and (current / 'data').exists():
        project_root = current
        break
    current = current.parent
else:
    # Fallback: assume we're in notebooks/ subdirectory
    project_root = notebook_dir.parent if notebook_dir.name == 'notebooks' else notebook_dir

# Add to path if not already there
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"Project root: {project_root}")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Project imports
from src.analysis.results_analyzer import ResultsAnalyzer, analyze_experiments
from src.analysis.visualization_utils import ThesisPlotter

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Imports successful!")

## 1. Load Experimental Results

In [ ]:
# Initialize results analyzer
analyzer = analyze_experiments(results_dir='../results')

print(f"Loaded {len(analyzer.experiments)} experiments:")
for exp_name in analyzer.experiments.keys():
    print(f"  - {exp_name}")

## 2. Training Curves Comparison

In [ ]:
# Plot training curves for all experiments
fig = analyzer.plot_training_curves(
    metrics=['loss', 'perplexity'],
    save_path='../figures/training_curves.png'
)

plt.show()
print("✅ Training curves saved")

## 3. IndicGLUE Evaluation Comparison

In [ ]:
# Compare IndicGLUE performance
fig = analyzer.plot_evaluation_comparison(
    eval_type='indicglue',
    save_path='../figures/indicglue_comparison.png'
)

plt.show()
print("✅ IndicGLUE comparison saved")

## 4. MultiBLiMP Syntactic Evaluation

In [ ]:
# Compare MultiBLiMP performance
fig = analyzer.plot_evaluation_comparison(
    eval_type='multiblimp',
    save_path='../figures/multiblimp_comparison.png'
)

plt.show()
print("✅ MultiBLiMP comparison saved")

## 5. Morphological Probes Analysis

In [ ]:
# Compare morphological probe performance
fig = analyzer.plot_evaluation_comparison(
    eval_type='probes',
    save_path='../figures/morphological_probes_comparison.png'
)

plt.show()
print("✅ Morphological probes comparison saved")

## 6. Statistical Significance Testing

In [ ]:
# Compare two best models statistically
experiments = list(analyzer.experiments.keys())

if len(experiments) >= 2:
    exp1, exp2 = experiments[0], experiments[1]
    
    print(f"Comparing: {exp1} vs {exp2}\n")
    
    # Statistical comparison
    comparison = analyzer.compare_models_statistically(
        exp1, exp2,
        metric='accuracy',
        eval_type='indicglue'
    )
    
    print("📊 Statistical Comparison:")
    print(f"\nSummary Statistics:")
    print(f"  {exp1} mean: {comparison['summary']['exp1_mean']:.4f} ± {comparison['summary']['exp1_std']:.4f}")
    print(f"  {exp2} mean: {comparison['summary']['exp2_mean']:.4f} ± {comparison['summary']['exp2_std']:.4f}")
    print(f"  Difference: {comparison['summary']['difference']:.4f}")
    
    print(f"\nPaired t-test:")
    print(f"  p-value: {comparison['t_test']['p_value']:.4f}")
    print(f"  Significant: {'Yes ✓' if comparison['t_test']['significant'] else 'No ✗'}")
    
    print(f"\nWilcoxon test:")
    print(f"  p-value: {comparison['wilcoxon']['p_value']:.4f}")
    print(f"  Significant: {'Yes ✓' if comparison['wilcoxon']['significant'] else 'No ✗'}")
    
    print(f"\nEffect Size:")
    print(f"  Cohen's d: {comparison['effect_size']['cohens_d']:.4f}")
    print(f"  Interpretation: {comparison['effect_size']['interpretation']}")
    
    print(f"\nBootstrap 95% CI:")
    print(f"  [{comparison['bootstrap_ci']['lower']:.4f}, {comparison['bootstrap_ci']['upper']:.4f}]")
else:
    print("Need at least 2 experiments for comparison")

## 7. Generate LaTeX Tables for Thesis

In [ ]:
# Generate LaTeX table for IndicGLUE
latex_table = analyzer.generate_latex_table(
    eval_type='indicglue',
    metric='accuracy',
    caption='IndicGLUE Benchmark Results',
    label='tab:indicglue_results',
    save_path='../tables/indicglue_results.tex'
)

print("IndicGLUE LaTeX Table:")
print(latex_table)
print("\n✅ LaTeX table saved to tables/indicglue_results.tex")

In [ ]:
# Generate LaTeX table for MultiBLiMP
latex_table = analyzer.generate_latex_table(
    eval_type='multiblimp',
    metric='accuracy',
    caption='MultiBLiMP Syntactic Phenomena Results',
    label='tab:multiblimp_results',
    save_path='../tables/multiblimp_results.tex'
)

print("✅ MultiBLiMP LaTeX table saved")

## 8. Layer-wise Probe Visualization

In [ ]:
# Visualize layer-wise probe results
plotter = ThesisPlotter(style='thesis')

# Example: Case detection probe
# (Replace with actual data from your experiments)
layer_results = {i: 0.5 + 0.03 * i + np.random.normal(0, 0.02) 
                for i in range(13)}

fig = plotter.plot_layer_wise_probe_results(
    layer_results,
    probe_name='Case Detection',
    title='Layer-wise Case Detection Accuracy',
    save_path='../figures/layer_wise_case_probe.png'
)

plt.show()
print("✅ Layer-wise probe visualization saved")

## 9. Model Size vs Performance

In [ ]:
# Plot performance vs model size
# (Replace with actual data)
model_sizes = [50, 110, 350]  # Millions of parameters
accuracies = [0.72, 0.78, 0.82]
model_names = ['Tiny', 'Small', 'Medium']

fig = plotter.plot_performance_vs_model_size(
    model_sizes,
    accuracies,
    model_names,
    title='Model Size vs IndicGLUE Performance',
    save_path='../figures/model_size_vs_performance.png'
)

plt.show()
print("✅ Model size comparison saved")

## 10. Generate Individual Experiment Reports

In [ ]:
# Generate reports for each experiment
for exp_name in analyzer.experiments.keys():
    report = analyzer.generate_summary_report(
        exp_name,
        save_path=f'../reports/{exp_name}_report.md'
    )
    print(f"✅ Generated report for {exp_name}")

print("\nAll reports saved to reports/ directory")

## Summary

This notebook analyzed:
- ✅ Training curves and convergence
- ✅ Evaluation metrics across benchmarks
- ✅ Statistical significance of differences
- ✅ LaTeX tables for thesis
- ✅ Layer-wise probing analysis
- ✅ Model size comparisons

All figures and tables ready for thesis inclusion!